# Fueling Pricing Strategies with ML

## Intro
The purpose of this notebook is to serve as a simple non comprehensive guide on how to use mlFlow to register ML models that can then be served as an endpoint and use it to provide powerful ML capabilities to build real-time analytics applications.

## Building the model

We will define a custom model using the pyfunc model wrapper and the statsmodels library to build a linnear regression using the OLS method. With this we will get the best possible line fit between the price and total sales data points, so we can explain the relationship between both variables with the slope of the line. Then we will use that slope value to define a new variable called "price elasticity", which explains the product sales sensitivity to different price changes. A product elasticity will be defined as: 

<br/>

1.  \\( > 1  = \\) Elastic : Changes in prices have a significant change in demand 
1.  \\( < 1 = \\) Inelastic : Changes in prices don't have significant change in demand

<br/>
So let's have a look on how to build it with Python and serve the model over an endpoint so it can be called in real time and generate a new price elasticity and a predicted sales quantity

In [0]:
#Importing required libraries
import numpy as np
import pandas as pd
import mlflow

In [0]:
# define a custom model using the MlFlow pyfunc functionalities
class MyModel(mlflow.pyfunc.PythonModel):
    
    def predict(self, context, model_input):
        return self.my_custom_function(model_input)
        
    def my_custom_function(self, model_input):
        import json 
        import numpy as np
        import pandas as pd
        from pandas import json_normalize
        
        #transforming data from JSON to pandas dataframe

        data_frame = pd.json_normalize(model_input)
        data_frame = data_frame.explode(["event_time", "price", "total_sales"]).drop(["category_code", "brand", "revenue"], axis=1)
        data_frame = data_frame.reset_index(drop=True)
                
        # Calculating slope (Aka. price elasticity)
        slope = ( (data_frame.price*data_frame.total_sales).mean() - data_frame.price.mean()*data_frame.total_sales.mean() ) / ( (((data_frame.price)**2).mean()) - (data_frame.price.mean())**2)
        price_elasticity = (slope)*(data_frame.price.mean()/data_frame.total_sales.mean())

        return price_elasticity

In [0]:
#Logging model as a experiment  
my_model = MyModel()
with mlflow.start_run():
    model_info = mlflow.pyfunc.log_model(artifact_path="model", python_model=my_model)

In [0]:
# load the model
loaded_model = mlflow.pyfunc.load_model(model_uri=model_info.model_uri)